This workbook is a bit of quick walkthrough of how to use ghidra-bridge/ghidra-notebook. We'll write another one that actually walks through Ghidra use.

In [ ]:
# setup code
import ipywidgets as widgets

Once you've got the bridge connected, the flat API should be available in the globals. Let's see what program you've got open in Ghidra.

In [ ]:
print(currentProgram)

Let's do something a bit fancier. Let's get the list of functions in that program. I wonder how, though...

Let's start with a list of what's available on currentProgram:

In [ ]:
# dir will include the attributes and functions of the bridged object in the Ghidra python environment
dir(currentProgram)

getFunctionManager looks like it's interesting. Can we get some more info on that? 

In [ ]:
# help will attempt to get the Ghidra documentation if possible
help(currentProgram.getFunctionManager())

Great! getFunctions() sounds like what we want. Let's do it!

In [ ]:
%time list(currentProgram.getFunctionManager().getFunctions(True))

Wow, that was pretty slow - GhidraBridge has to pump multiple requests for each function, so it can take a while. Luckily, the bridge.remote_eval() function lets you run list/dictionary-comprehensions that can be a bit faster. We really just want the name of the function and the address, so let's try that.

In [ ]:
# the bridge is available in the globals as "bridge". 
# Note that remote_eval takes a string.
%time func_tuple_list = bridge.remote_eval("[(func.getEntryPoint().getOffset(), func.getName()) for func in currentProgram.getFunctionManager().getFunctions(True)]")
func_dict = { f"{name} ({hex(address)})": (address, name) for address, name in func_tuple_list}
func_dict

Yay, way faster. Now let's pick a function to look at with a widget:

In [ ]:
dropD = widgets.Dropdown(
 options=sorted(func_dict.keys()),
 description="Function name:",
 disabled=False,
 style={'description_width': 'initial'}
 )
display(dropD)

Let's get the decompilation of the function.

In [ ]:
def getAddress(program, hex_address):
    return program.parseAddress(hex_address)[0]    

def getFunctionByEntrypoint(program, entrypoint_address):
    return program.getFunctionManager().getFunctionAt(entrypoint_address)

# note that you can just remotely import across the bridge
# imports will try to fulfill locally first, then check a bridge with the import hook set to try to find something that can fulfill it remotely.
from ghidra.app.decompiler import DecompInterface
from ghidra.util.task import ConsoleTaskMonitor

def getFunctionDecompile(function):
    ifc = DecompInterface()
    ifc.openProgram(function.getProgram())

    # decompile the function and print the pseudo C
    results = ifc.decompileFunction(function, 0, ConsoleTaskMonitor())
    return results.getDecompiledFunction().getC()

function_address = hex(func_dict[dropD.value][0])
print(getFunctionDecompile(getFunctionByEntrypoint(currentProgram, getAddress(currentProgram, function_address))))


Hmm, can we be even more interactive? What about automatically updating the decompile when the user changes the selection? And also jumping Ghidra to that location?!

In [ ]:
dropD2 = widgets.Dropdown(
 options=sorted(func_dict.keys()),
 description="Function name:",
 disabled=False,
 style={'description_width': 'initial'}
 )
output = widgets.Output()

display(dropD2, output)

def on_value_change(change):
    # callback handler
    output.clear_output()
    with output:
        function_address = hex(func_dict[change['new']][0])
        addr = getAddress(currentProgram, function_address)
        print(getFunctionDecompile(getFunctionByEntrypoint(currentProgram, addr)))
        state.setCurrentAddress(addr)
        print("!! Go look at this in your Ghidra window!")

# register the callback to update when the user changes the value in the dropdown
dropD2.observe(on_value_change, names='value')

# trigger on_value_change to the first value, as if the user had selected it alread
on_value_change({"new": dropD2.options[0]})